In [1]:
import requests
import wptools
import re
import pandas as pd
import os
import csv
import json
from deep_translator import GoogleTranslator
import urllib3
from bs4 import BeautifulSoup

In [2]:
page = wptools.page('Mountain_War_(Lebanon)')
page.get_parse()

en.wikipedia.org (parse) Mountain_War_(Lebanon)
Mountain War (Lebanon) (en) data
{
  infobox: <dict(11)> conflict, partof, date, place, casus, result...
  iwlinks: <list(1)> https://fr.wikipedia.org/wiki/Kfarmatta
  pageid: 30729738
  parsetree: <str(117969)> <root><template><title>redirect</title>...
  requests: <list(1)> parse
  title: Mountain War (Lebanon)
  wikibase: Q7784981
  wikidata_url: https://www.wikidata.org/wiki/Q7784981
  wikitext: <str(99290)> {{redirect|Mountain War|other uses|mounta...
}


In [3]:
session = requests.Session()

url = "https://en.wikipedia.org/w/api.php"
params = {
    "action": "query",
    "format": "json",
    "titles": "Mountain_War_(Lebanon)",
    "prop": "links",
    "pllimit": "max"
}

page = wptools.page('Mountain War (Lebanon)')

response = session.get(url=url, params=params)
data = response.json()
pages = data["query"]["pages"]

pg_count = 1
page_titles = []

print("Page %d" % pg_count)
for key, val in pages.items():
    for link in val["links"]:
        print(link["title"])
        page_titles.append(link["title"])

while "continue" in data:
    plcontinue = data["continue"]["plcontinue"]
    params["plcontinue"] = plcontinue

    response = session.get(url=url, params=params)
    data = response.json()
    pages = data["query"]["pages"]

    pg_count += 1

    print("\nPage %d" % pg_count)
    for key, val in pages.items():
        for link in val["links"]:
            print(link["title"])
            page_titles.append(link["title"])

print("%d titles found." % len(page_titles))

Page 1
10th Airmobile Brigade (Lebanon)
10th Infantry Brigade (Lebanon)
11th Infantry Brigade (Lebanon)
1860 Mount Lebanon civil war
1978 South Lebanon conflict
1982 Beirut bombing
1982 Iranian diplomats kidnapping
1982 Lebanon War
1983 Beirut barracks bombing
1983 Beirut barracks bombings
1983 United States embassy bombing
1983 United States embassy bombing in Beirut
1984 Sohmor massacre
1984 United States embassy annex bombing in Beirut
1985 Beirut car bombings
24th Marine Expeditionary Unit
37 mm automatic air defense gun M1939 (61-K)
3rd Infantry Brigade (Lebanon)
4th Infantry Brigade (Lebanon)
5th Infantry Brigade (Lebanon)
6th Infantry Brigade (Lebanon)
76 mm divisional gun M1942 (ZiS-3)
7th Infantry Brigade (Lebanon)
8th Infantry Brigade (Lebanon)
9K31 Strela-1
9K32 Strela-2
9th Infantry Brigade (Lebanon)
AZP S-60
Aabey
Achrafieh
Ain Ksour
Air defense
Aircraft carrier
Al-Mourabitoun
Al Bennay
Aley
Aley District
Amin Gemayel
Amin Qadi
Amine Gemayel
Anti-aircraft
Anti-aircraft war

In [4]:
def get_arabic_name(english_phrase):
    if GoogleTranslator(source='auto', target='ar').translate(english_phrase) == english_phrase:
        print('Cannot translate phrase `{}` to arabic'.format(english_phrase))
        return -1, -1

    response = requests.get('http://en.wikipedia.org/wiki/{}'.format(english_phrase))
    if response.status_code != 200:
        print('Cannot find the link: {}'.format('http://en.wikipedia.org/wiki/{}'.format(english_phrase)))
        return GoogleTranslator(source='auto', target='ar').translate(english_phrase) + ' (مترجمه)', -1

    http = urllib3.PoolManager()
    url = 'http://en.wikipedia.org/wiki/{}'.format(english_phrase)
    response = http.request('GET', url)

    # get languages and links
    soup = BeautifulSoup(response.data)
    links = [(el.get('lang'), el.get('href')) for el in soup.select('li.interlanguage-link > a')]

    if links != []:
        for language, link in links:
            if language == 'ar':
                response = http.request('GET', link)
                soup = BeautifulSoup(response.data)

                arabic_phrase = soup.title.text
                arabic_phrase = re.sub('- ويكيبيديا', '', arabic_phrase)
                return arabic_phrase, link

    return GoogleTranslator(source='auto', target='ar').translate(english_phrase) + ' (مترجمه)', -1

def remove_file_instances(s):
    if '[[' in s and ']]' in s and 'File' in s:
        ind2 = s.rfind(']')
        return s[ind2+1:]
    elif '[[' in s and ']]' in s and 'Image' in s:
        ind2 = s.rfind(']')
        return s[ind2+1:]
    else:
        return s

# def to_delete(s):
#     return 'name' in s or '{' in s or '}' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s

def to_delete(s):
    # return 'name' in s or '{' in s or '}' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s or 'Historical' in s or 'date' in s or len(s) == 1 or '\"' in s or '(' in s or 'not in' in s or 'Nowrap' in s
    return 'name' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s or 'Historical' in s or 'Current' in s or 'Formerly' in s or 'date' in s or len(s) == 1 or '\"' in s or '(' in s or 'not in' in s or 'Nowrap' in s or 'cite' in s or 'citation' in s or 'Cite' in s or 'http' in s

def get_list_items(s):
    # s = re.sub('{{.*?}}', '', s)
    # print(1, s)
    s = re.sub('\'', '', s)
#     print(2, s)
    s = re.sub(r'\[\[(?:[^\]|]*\|)?([^\]|]*)\]\]', r'\1', s)
#     print(3, s)
    s = re.sub('\n----', '<br>', s)
#     print(4, s)
    s = re.sub('\n\*', '<br>', s)
#     print(5, s)
    s = re.sub('<br />', '<br>', s)
#     print(6, s)
    s = re.sub('<br/>', '<br>', s)
#     print(7, s)
    s = re.sub('</ref>|ref||</ref>', '', s)
#     print(8, s)
    s = re.sub('</ref>', '', s)
#     print(9, s)
    s = re.sub('ref', '', s)
#     print(10, s)
#     s = re.sub('|', '', s)
    s = s.translate({ord('|'): '<br>'})
#     print(11, s)
    s = s.translate({ord('{'): None, ord('}'): None})
#     print(11, s)
#     if '<br>' in s:
    s = s.split("<br>")
#     print(12, s)
#     else:
#         s = s.split("\|")
    s = [e.strip() for e in s if e.strip() != '']
#     print(13, s)
    s = [remove_file_instances(e).strip() for e in s]
#     print(14, s)
    s = [e for e in s if not to_delete(e)]
#     print(15,  s)
    s = list(set(s)) # keep unique entries
    
    s = [e for e in s if not e.isdigit()] # remove entries that are just numbers
#     print('s is now: ', s)
    if len(s) > 1:
        new_s = []
        for e in s:
            if ',' in e:
                splitted_e = e.split(',')
                for ee in splitted_e:
                    new_s.append(ee)
            elif 'to' in e:
                splitted_e = e.split('to')
                for ee in splitted_e:
                    new_s.append(ee)
            else:
                new_s.append(e)
    #     print('s is now after: ', new_s)
        new_s = [e.strip() for e in new_s if e.strip() != '']
        return new_s
    else:
        return s




In [5]:
def mkdir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

In [6]:
df_political_party_en = pd.DataFrame(columns=['Name', 'Ideology', 'Political Position', 'Founder(s)'])
df_political_party_ar = pd.DataFrame(columns=['الاسم','الإيديولوجيا', 'الموقف السياسي', 'المؤسس (المؤسسون)'])

In [7]:
for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'ideology' in page.data['infobox']:
                    print(page)

                    leaders_types = [k for k in page.data['infobox'].keys() if 'leader' in k]
                    if leaders_types == []:
                        pass
                    else:
                        founders = ''
                        count = 0
                        for lt in leaders_types:
                            if count%2 == 0:
                                pass
#                                 founders += ''.join(get_list_items(page.data['infobox'][lt]))+ ':'
                            else:
                                print(page.data['infobox'][lt])
                                founders += ','.join(get_list_items(page.data['infobox'][lt])) + ','
                            count += 1
                    
                    name = get_list_items(page.data['infobox']['name']) if 'name' in page.data['infobox'] else '-'
                    if isinstance(name, list):
                        if len(name) > 1:
                            for n in name:
                                print('name before: ', n)
                                name = [n]
                                break
#                             name = list(name[0])
                            print('name after: ', name)
                    ideology = get_list_items(page.data['infobox']['ideology'])
                    position = get_list_items(page.data['infobox']['position']) if 'position' in page.data['infobox'] else '-'
                    
                    df_political_party_en = df_political_party_en.append({
                        'Name': ','.join(name if isinstance(name, list) else name),
                        'Ideology': ','.join(ideology) if isinstance(ideology, list) else ideology,
                        'Political Position': ','.join(position) if isinstance(position, list) else position,
                        'Founder(s)': founders,
                    }, ignore_index=True)
                    
                    print('-----------------')
                    print('name: ', name)
                    print('founders: ', founders)
                    print('ideology: ', ideology)
                    print('position: ', position)
                    
                    
                    
                    # get the arabic name, ideology, political position, and founders of the political party
#                     page_ar = wptools.page(p, lang='ar')
#                     page_ar.get_parse()
#                     name_ar = page_ar.data['title']
                    
                    if name != '':
                        print('START')
                        name_ar = []
                        for n in name:
                            n_ar, n_ar_link = get_arabic_name(n)
                            if n_ar != -1:
                                print('name: ', n)
                                print('name ar: ', n_ar),
                                print('name ar link: ', n_ar_link)
                                name_ar.append(n_ar)
                            else:
                                continue
                        print('END')
                    else:
                        name_ar = '-'
                    
                    print('START')
                    ideology_ar = []
                    for ideo in ideology:
                        ideo_ar, ideo_ar_link = get_arabic_name(ideo)
                        if ideo_ar != -1:
                            print('ideology: ', ideo)
                            print('ideology ar: ', ideo_ar),
                            print('ideo ar link: ', ideo_ar_link)
                            ideology_ar.append(ideo_ar)
                        else:
                            continue
                    print('END')
                    
                    if position != '-':
                        position_ar = []
                        print('START')
                        for pos in position:
                            pos_ar, pos_ar_link = get_arabic_name(pos)
                            if pos_ar != -1:
                                print('position: ', pos)
                                print('position ar: ', pos_ar),
                                print('position ar link: ', pos_ar_link)
                                position_ar.append(pos_ar)
                            else:
                                continue
                        print('END')
                    else:
                        position_ar = '-'
                    
                    print('START')
                    founders_ar = []
                    for f in founders.split(','):
                        if f.strip() != '':
                            f_ar, f_ar_link = get_arabic_name(f)
                            if f_ar != -1:
                                print('founder: ', f)
                                print('founder ar: ', f_ar),
                                print('founder ar link: ', f_ar_link)
                                founders_ar.append(f_ar)
                            else:
                                continue
                    print('END')
                    
                    
                    df_political_party_ar = df_political_party_ar.append({
                        'الاسم': ','.join(name_ar),
                        'الإيديولوجيا': ','.join(ideology_ar),
                        'الموقف السياسي': ','.join(position_ar),
                        'المؤسس (المؤسسون)': ','.join(founders_ar),
                    }, ignore_index=True)
                    
        
        except LookupError:
            continue
                
            
    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 10th Airmobile Brigade (Lebanon)
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) 10th Infantry Brigade (Lebanon)
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) 11th Infantry Brigade (Lebanon)
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wik

-----------------
name:  []
founders:  
ideology:  ['Pan-Arabism', 'Anti-imperialism', 'Arab socialism', 'Arab nationalism', 'Nasserism']
position:  ['Left-wing']
START
END
START
ideology:  Pan-Arabism
ideology ar:  وحدة عربية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%88%D8%AD%D8%AF%D8%A9_%D8%B9%D8%B1%D8%A8%D9%8A%D8%A9
ideology:  Anti-imperialism
ideology ar:  معاداة الاستعمارية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%85%D8%B9%D8%A7%D8%AF%D8%A7%D8%A9_%D8%A7%D9%84%D8%A7%D8%B3%D8%AA%D8%B9%D9%85%D8%A7%D8%B1%D9%8A%D8%A9
ideology:  Arab socialism
ideology ar:  اشتراكية عربية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D8%A7%D8%B4%D8%AA%D8%B1%D8%A7%D9%83%D9%8A%D8%A9_%D8%B9%D8%B1%D8%A8%D9%8A%D8%A9
ideology:  Arab nationalism
ideology ar:  قومية عربية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%82%D9%88%D9%85%D9%8A%D8%A9_%D8%B9%D8%B1%D8%A8%D9%8A%D8%A9
ideology:  Nasserism
ideology ar:  التيار الناصري 
ideo ar link:  https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AA%D9

en.wikipedia.org (parse) Al Bennay
en.wikipedia.org (parse) Aley
en.wikipedia.org (parse) Aley District
en.wikipedia.org (parse) Amin Gemayel
en.wikipedia.org (imageinfo) File:C10756-9A (cropped).jpg
en.wikipedia.org (parse) Amin Qadi
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Amine Gemayel
en.wikipedia.org (imageinfo) File:C10756-9A (cropped).jpg
en.wikipedia.org (parse) Anti-aircraft
en.wikipedia.org (parse) Anti-aircraft warfare
en.wikipedia.org (parse) Arabic
en.wikipedia.org (imageinfo) File:Arabic albayancalligraphy.svg
en.wikipedia.org (parse) Arabic language
en.wikipedia.org (imageinfo) File:Arabic albayancalligraphy.svg
en.wikipedia.org (parse)

-----------------
name:  ['As-Saiqa']
founders:  
ideology:  ['Baathism', 'Pan-Arabism']
position:  -
START
name:  As-Saiqa
name ar:  منظمة الصاعقة 
name ar link:  https://ar.wikipedia.org/wiki/%D9%85%D9%86%D8%B8%D9%85%D8%A9_%D8%A7%D9%84%D8%B5%D8%A7%D8%B9%D9%82%D8%A9
END
START
ideology:  Baathism
ideology ar:  بعثية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D8%A8%D8%B9%D8%AB%D9%8A%D8%A9
ideology:  Pan-Arabism
ideology ar:  وحدة عربية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%88%D8%AD%D8%AF%D8%A9_%D8%B9%D8%B1%D8%A8%D9%8A%D8%A9
END
START
END


en.wikipedia.org (parse) Assassination of Bachir Gemayel
en.wikipedia.org (imageinfo) File:Bachir With His Father Pierre (...
en.wikipedia.org (parse) Assassination of René Moawad
en.wikipedia.org (imageinfo) File:René Moawad.jpg
en.wikipedia.org (parse) At Tiri incident
en.wikipedia.org (parse) Attack helicopter
en.wikipedia.org (parse) Autocannons
en.wikipedia.org (parse) Awali River
en.wikipedia.org (imageinfo) File:Awali River Route.png
en.wikipedia.org (parse) Aytat
en.wikipedia.org (parse) BGM-71 TOW
en.wikipedia.org (imageinfo) File:Hires 090509-A-4842R-001a.jpg
en.wikipedia.org (parse) BTR-152
en.wikipedia.org (imageinfo) File:BTR-152 armoured transporterr a...
en.wikipedia.org (parse) Baabda
en.wikipedia.org (parse) Baabda District
en.wikipedia.org (parse) Baabda Palace
en.wikipedia.org (parse) Baalbek
en.wikipedia.org (parse) Baalechmay
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for AP

...

-----------------
name:  ['Progressive Socialist Party']
founders:  
ideology:  ['Third Way', 'Pan-Arabism', 'Social democracy', 'Democratic socialism', 'Arab nationalism']
position:  ['Left-wing', 'Centre', 'centre-left']
START
name:  Progressive Socialist Party
name ar:  الحزب التقدمي الاشتراكي 
name ar link:  https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B2%D8%A8_%D8%A7%D9%84%D8%AA%D9%82%D8%AF%D9%85%D9%8A_%D8%A7%D9%84%D8%A7%D8%B4%D8%AA%D8%B1%D8%A7%D9%83%D9%8A
END
START
ideology:  Third Way
ideology ar:  طرف خارجي 
ideo ar link:  https://ar.wikipedia.org/wiki/%D8%B7%D8%B1%D9%81_%D8%AE%D8%A7%D8%B1%D8%AC%D9%8A
ideology:  Pan-Arabism
ideology ar:  وحدة عربية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%88%D8%AD%D8%AF%D8%A9_%D8%B9%D8%B1%D8%A8%D9%8A%D8%A9
ideology:  Social democracy
ideology ar:  ديمقراطية اجتماعية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D8%AF%D9%8A%D9%85%D9%82%D8%B1%D8%A7%D8%B7%D9%8A%D8%A9_%D8%A7%D8%AC%D8%AA%D9%85%D8%A7%D8%B9%D9%8A%D8%A9
ideology:  Demo

en.wikipedia.org (parse) Public domain
en.wikipedia.org (parse) RAF Akrotiri
en.wikipedia.org (imageinfo) File:RAF CONDUCTS FIRST AIR STRIKES ...
en.wikipedia.org (parse) RPG-7
en.wikipedia.org (imageinfo) File:RPG-7 detached.jpg
en.wikipedia.org (parse) Ramlet al-Baida
en.wikipedia.org (parse) Raouché
en.wikipedia.org (parse) Ras el-Ain
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Ras el-Matn
en.wikipedia.org (parse) Ras el Haref
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wiki

Rabi Banat
[[Antun Saadeh]]
-----------------
name:  ['Syrian Social Nationalist Party']
founders:  Rabi Banat,Antun Saadeh,
ideology:  ['Pan-Syrianism', 'ubl', 'Syrian nationalism', 'Corporatism', 'Anti-Zionism', 'Secularism', 'Anti-communism Page needed', 'officially denied)', 'Syrian irredentism', 'officially denied) collapsible list', 'Page needed', 'August 2021', 'Social nationalism']
position:  ['Left-wing', 'Syncretic']
START
name:  Syrian Social Nationalist Party
name ar:  الحزب السوري القومي الاجتماعي 
name ar link:  https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B2%D8%A8_%D8%A7%D9%84%D8%B3%D9%88%D8%B1%D9%8A_%D8%A7%D9%84%D9%82%D9%88%D9%85%D9%8A_%D8%A7%D9%84%D8%A7%D8%AC%D8%AA%D9%85%D8%A7%D8%B9%D9%8A
END
START
ideology:  Pan-Syrianism
ideology ar:  بلاد الشام 
ideo ar link:  https://ar.wikipedia.org/wiki/%D8%A8%D9%84%D8%A7%D8%AF_%D8%A7%D9%84%D8%B4%D8%A7%D9%85
Cannot translate phrase `ubl` to arabic
ideology:  Syrian nationalism
ideology ar:  القومية السورية 
ideo ar link:  

en.wikipedia.org (parse) Syrian intervention in the Lebanese Civil War
en.wikipedia.org (parse) Syrian occupation of Lebanon
en.wikipedia.org (imageinfo) File:Cedar Revolution Demonstrators.jpg
en.wikipedia.org (parse) T-54/55
en.wikipedia.org (imageinfo) File:Victory park (Kazan) (262-14).jpg
en.wikipedia.org (parse) TIME Magazine
en.wikipedia.org (parse) Tactical Airborne Reconnaissance Pod System
en.wikipedia.org (parse) Technical (vehicle)
en.wikipedia.org (parse) The New York Times
en.wikipedia.org (imageinfo) File:The-New-York-Times-March-26-2018.jpg
en.wikipedia.org (parse) Ti-67
en.wikipedia.org (parse) Training aircraft
en.wikipedia.org (parse) Tripoli, Lebanon
en.wikipedia.org (parse) Tyre, Lebanon
en.wikipedia.org (parse) U.S. Marines
en.wikipedia.org (imageinfo) File:Emblem of the United States Mar...
en.wikipedia.org (parse) UPI
en.wikipedia.org (parse) USS Arthur W. Radford (DD-968)
en.wikipedia.org (parse) USS Bowen (FF-1079)
en.wikipedia.org (parse) USS Independence (CV

skipping Template:Campaignbox Lebanese Civil War
skipping Template:Country data France
skipping Template:Country data Lebanon
skipping Template:Country data PLO
skipping Template:Country data Syria
skipping Template:Country data USA
skipping Template:Country data United States
skipping Template talk:Campaignbox Lebanese Civil War
skipping Category:Use dmy dates from July 2017


In [8]:
df_political_party_en

,Name,Ideology,Political Position,Founder(s)
0,,"Pan-Arabism,Anti-imperialism,Arab socialism,Ar...",Left-wing,
1,As-Saiqa,"Baathism,Pan-Arabism",-,
2,Fatah,"Secularism,Social democracy,Palestinian nation...","Centre-left,left-wing","Mahmoud Aloul,"
3,Lebanese Kataeb Party,"Anti-communism,Social conservatism,Lebanese na...","Right-wing,far-right,Centre-right","Samy Gemayel,Pierre Gemayel,"
4,Lebanese Communist Party,"Marxism–Leninism,Communism",Far-left,
5,Lebanese Forces,"Christian nationalism,Lebanese nationalism,Lib...",Right-wing,"Samir Geagea,Bachir Gemayel,"
6,القوات اللبنانية,"Christian nationalism,Conservatism",-,
7,National Liberal Party,"Lebanese nationalism,National liberalism,Econo...",Centre-right,"Camille Dory Chamoun,Robert Khoury,Pierre Gear..."
8,Palestine Liberation Organization,Palestinian nationalism,-,"Mahmoud Abbas,"
9,Popular Front for the Liberation of Palestine,"Pan-Arabism,ubl,Anti-Zionism,Secularism,Palest...",Far-left,"Ahmad Saadat,"


In [9]:
df_political_party_ar

,الاسم,الإيديولوجيا,الموقف السياسي,المؤسس (المؤسسون)
0,,"وحدة عربية ,معاداة الاستعمارية ,اشتراكية عربية...",يسارية,
1,منظمة الصاعقة,"بعثية ,وحدة عربية",-,
2,حركة فتح,"علمانية ,ديمقراطية اجتماعية ,قومية فلسطينية","اليسار المعتدل ,يسارية",
3,حزب الكتائب اللبنانية (مترجمه),"معاداة الشيوعية ,محافظة اجتماعية ,قومية لبناني...","يمينية ,يمين متطرف ,وسط اليمين","سامي الجميل ,بيار الجميل"
4,الحزب الشيوعي اللبناني,"ماركسية لينينية ,شيوعية",أقصى اليسار,
5,القوات اللبنانية,"قومية مسيحية ,قومية لبنانية ,محافظة ليبرالية ,...",يمينية,"سمير جعجع ,بشير الجميل"
6,,"قومية مسيحية ,سياسة محافظة",-,
7,الحزب الوطني الليبرالي (مترجمه),"قومية لبنانية ,ليبرالية وطنية ,ليبرالية اقتصادية",وسط اليمين,"كميل دوري شمعون (مترجمه),روبرت خوري (مترجمه),ب..."
8,منظمة التحرير الفلسطينية,قومية فلسطينية,-,
9,الجبهة الشعبية لتحرير فلسطين,"وحدة عربية ,معاداة الصهيونية ,علمانية ,قومية ف...",أقصى اليسار,


In [10]:
datasets_dir = 'datasets_updated/1982-1984/Mountain_War_(Lebanon)/'

In [11]:
mkdir(datasets_dir)
df_political_party_en.to_csv(os.path.join(datasets_dir, 'political_parties_en.csv'), index=False)
df_political_party_ar.to_csv(os.path.join(datasets_dir, 'political_parties_ar.csv'), index=False, encoding='utf-8-sig')

In [12]:
df_politician_en = pd.DataFrame(columns=['Name', 'Political Party', 'Nationality', 'Religion'])
df_politician_ar = pd.DataFrame(columns=['الدين'       , 'الجنسيه'      , 'الحزب السياسي' , 'الاسم'])

In [13]:
for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'birth_date' in page.data['infobox']:
                    print(page)

                    name = get_list_items(page.data['infobox']['name']) if 'name' in page.data['infobox'] else '-'
                    native_name = get_list_items(page.data['infobox']['native_name']) if 'native_name' in page.data['infobox'] else '-'
                    nickname = get_list_items(page.data['infobox']['nickname']) if 'nickname' in page.data['infobox'] else '-'
                
                    religion = get_list_items(page.data['infobox']['religion']) if 'religion' in page.data['infobox'] else '-'
                    party = get_list_items(page.data['infobox']['party']) if 'party' in page.data['infobox'] else '-'
                    nationality = get_list_items(page.data['infobox']['nationality']) if 'nationality' in page.data['infobox'] else '-'
                    
                    if name == '-':
                        name_inserted = native_name
                    else:
                        name_inserted = name
                    name_inserted = ','.join(name_inserted) if isinstance(name_inserted, list) else name_inserted,
                    
                    df_politician_en = df_politician_en.append({
                        'Name': name_inserted,
                        'Political Party': ','.join(party) if isinstance(party, list) else party,
                        'Nationality': ','.join(nationality) if isinstance(nationality, list) else nationality,
                        'Religion': ','.join(religion) if isinstance(religion, list) else religion,
                    }, ignore_index=True)

                    print('-----------------')
                    print('name: ', name)
                    print('native_name: ', native_name)
                    print('nickname: ', nickname)
                    print('religion: ', religion)
                    print('party: ', party)
                    print('nationality: ', nationality)
                    
                    # get name from wikipedia
                    # get political party from wikipedia
                    # translate nationality
                    # translate religion
                    name_ar, name_ar_link = get_arabic_name(p)
                    print('name ar: ', name_ar)
                    
                    if party != '-':
                        print('START')
                        political_party_ar = []
                        for p in party:
                            p_ar, p_ar_link = get_arabic_name(p)
                            if p_ar != -1:
                                print('political party: ', p)
                                print('political party ar: ', p_ar),
                                print('political party ar link: ', p_ar_link)
                                political_party_ar.append(p_ar)
                            else:
                                continue
                        print('END')
                    else:
                        political_party_ar = '-'
                    
                    if nationality != '-':
                        print('START')
                        nationality_ar = []
                        for n in nationality:
                            n_ar, n_ar_link = get_arabic_name(n)
                            if n_ar != -1:
                                print('nationality: ', n)
                                print('nationality ar: ', n_ar),
                                print('nationality ar link: ', n_ar_link)
                                nationality_ar.append(n_ar)
                            else:
                                continue
                        print('END')
                    else:
                        nationality_ar = '-'
                        
                    if religion != '-':
                        print('START')
                        religion_ar = []
                        for r in religion:
                            r_ar, r_ar_link = get_arabic_name(r)
                            if r_ar != -1:
                                print('religion: ', r)
                                print('religion ar: ', r_ar),
                                print('religion ar link: ', r_ar_link)
                                religion_ar.append(r_ar)
                            else:
                                continue
                        print('END')
                    else:
                        religion_ar = '-'
                    
                    df_politician_ar = df_politician_ar.append({
                        'الاسم': ','.join(name_ar) if isinstance(name_ar, list) and len(name_ar) > 1 else name_ar,
                        'الحزب السياسي': ','.join(political_party_ar) if isinstance(political_party_ar, list) else political_party_ar,
                        'الجنسيه': ','.join(nationality_ar) if isinstance(nationality_ar, list) else nationality_ar,
                        'الدين': ','.join(religion_ar) if isinstance(religion_ar, list) else religion_ar,
                    }, ignore_index=True)
                    
                    print('name ar: ', name_ar)
                    print('religion ar: ', political_party_ar)
                    print('party ar: ', party)
                    print('nationality ar: ', nationality_ar)

        
        except LookupError:
            continue
                
            
    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 10th Airmobile Brigade (Lebanon)
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) 10th Infantry Brigade (Lebanon)
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) 11th Infantry Brigade (Lebanon)
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wik

-----------------
name:  ['Amine Gemayel']
native_name:  ['أمين الجميل']
nickname:  -
religion:  -
party:  ['Kataeb Party']
nationality:  ['Lebanese']
name ar:  أمين الجميل 
START
political party:  Kataeb Party
political party ar:  حزب الكتائب اللبنانية 
political party ar link:  https://ar.wikipedia.org/wiki/%D8%AD%D8%B2%D8%A8_%D8%A7%D9%84%D9%83%D8%AA%D8%A7%D8%A6%D8%A8_%D8%A7%D9%84%D9%84%D8%A8%D9%86%D8%A7%D9%86%D9%8A%D8%A9
END
START
nationality:  Lebanese
nationality ar:  لبناني (مترجمه)
nationality ar link:  -1
END
name ar:  أمين الجميل 
religion ar:  ['حزب الكتائب اللبنانية ']
party ar:  ['Kataeb Party']
nationality ar:  ['لبناني (مترجمه)']


en.wikipedia.org (parse) Amin Qadi
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Amine Gemayel
en.wikipedia.org (imageinfo) File:C10756-9A (cropped).jpg


-----------------
name:  ['Amine Gemayel']
native_name:  ['أمين الجميل']
nickname:  -
religion:  -
party:  ['Kataeb Party']
nationality:  ['Lebanese']
name ar:  أمين الجميل 
START
political party:  Kataeb Party
political party ar:  حزب الكتائب اللبنانية 
political party ar link:  https://ar.wikipedia.org/wiki/%D8%AD%D8%B2%D8%A8_%D8%A7%D9%84%D9%83%D8%AA%D8%A7%D8%A6%D8%A8_%D8%A7%D9%84%D9%84%D8%A8%D9%86%D8%A7%D9%86%D9%8A%D8%A9
END
START
nationality:  Lebanese
nationality ar:  لبناني (مترجمه)
nationality ar link:  -1
END
name ar:  أمين الجميل 
religion ar:  ['حزب الكتائب اللبنانية ']
party ar:  ['Kataeb Party']
nationality ar:  ['لبناني (مترجمه)']


en.wikipedia.org (parse) Anti-aircraft
en.wikipedia.org (parse) Anti-aircraft warfare
en.wikipedia.org (parse) Arabic
en.wikipedia.org (imageinfo) File:Arabic albayancalligraphy.svg
en.wikipedia.org (parse) Arabic language
en.wikipedia.org (imageinfo) File:Arabic albayancalligraphy.svg
en.wikipedia.org (parse) Aramoun, Aley
en.wikipedia.org (parse) Arbaniyeh
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Ariel Sharon
en.wikipedia.org (imageinfo) File:Ariel Sharon Headshot.jpg


...

-----------------
name:  ['Nobold', 'شفيق الوزان lang', 'شفيق الوزان', 'ar', 'Shafik Wazzan', 'lang']
native_name:  -
nickname:  -
religion:  -
party:  ['Independent']
nationality:  ['Lebanese']
name ar:  شفيق الوزان 
START
political party:  Independent
political party ar:  مستقل (توضيح) 
political party ar link:  https://ar.wikipedia.org/wiki/%D9%85%D8%B3%D8%AA%D9%82%D9%84_(%D8%AA%D9%88%D8%B6%D9%8A%D8%AD)
END
START
nationality:  Lebanese
nationality ar:  لبناني (مترجمه)
nationality ar link:  -1
END
name ar:  شفيق الوزان 
religion ar:  ['مستقل (توضيح) ']
party ar:  ['Independent']
nationality ar:  ['لبناني (مترجمه)']


en.wikipedia.org (parse) Shahhar
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Shatila refugee camp
en.wikipedia.org (parse) Shia Islam in Lebanon
en.wikipedia.org (imageinfo) File:LebanonShia.png
en.wikipedia.org (parse) Sidon
en.wikipedia.org (parse) Siege of Tel al-Zaatar
en.wikipedia.org (imageinfo) File:JeanJacquesKurz-DestructionOfTe...
en.wikipedia.org (parse) Sin el Fil
en.wikipedia.org (parse) Sodeco Square
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postor

-----------------
name:  ['Walid Jumblatt']
native_name:  ['lang', 'وليد جنبلاط', 'ar']
nickname:  -
religion:  -
party:  ['Progressive Socialist Party']
nationality:  -
name ar:  وليد جنبلاط 
START
political party:  Progressive Socialist Party
political party ar:  الحزب التقدمي الاشتراكي 
political party ar link:  https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B2%D8%A8_%D8%A7%D9%84%D8%AA%D9%82%D8%AF%D9%85%D9%8A_%D8%A7%D9%84%D8%A7%D8%B4%D8%AA%D8%B1%D8%A7%D9%83%D9%8A
END
name ar:  وليد جنبلاط 
religion ar:  ['الحزب التقدمي الاشتراكي ']
party ar:  ['Progressive Socialist Party']
nationality ar:  -


en.wikipedia.org (parse) War Powers Resolution
en.wikipedia.org (parse) War of Brothers
en.wikipedia.org (parse) War of Liberation (1989–1990)
en.wikipedia.org (parse) War of the Camps
en.wikipedia.org (parse) Weapons of the Lebanese Civil War
en.wikipedia.org (parse) West Beirut
en.wikipedia.org (parse) Yarze
en.wikipedia.org (parse) Yazbaki
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Youssef Tahan
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/media

skipping Template:Campaignbox Lebanese Civil War
skipping Template:Country data France
skipping Template:Country data Lebanon
skipping Template:Country data PLO
skipping Template:Country data Syria
skipping Template:Country data USA
skipping Template:Country data United States
skipping Template talk:Campaignbox Lebanese Civil War
skipping Category:Use dmy dates from July 2017


In [14]:
df_politician_en

,Name,Political Party,Nationality,Religion
0,"(Amine Gemayel,)",Kataeb Party,Lebanese,-
1,"(Amine Gemayel,)",Kataeb Party,Lebanese,-
2,"(-,)",plainlist,-,-
3,"(Bachir Gemayel,lang,بشير الجميّل,ar,)",Kataeb Party,-,-
4,"(رينيه معوض lang,رينيه معوض,ar,René Moawad<Br/...",Independent,-,-
5,"(Bachir Gemayel,lang,بشير الجميّل,ar,)",Kataeb Party,-,-
6,"(Robert O. Goodman,)",-,-,-
7,"(Elie Hobeika,)",-,Lebanese,-
8,"(الأمير مجيد توفيق أرسلان Lang,Majid Toufic Ar...",-,-,-
9,"(فادي أفرام,Fadi Frem,)",-,-,-


In [15]:
df_politician_ar

,الدين,الجنسيه,الحزب السياسي,الاسم
0,-,لبناني (مترجمه),حزب الكتائب اللبنانية,أمين الجميل
1,-,لبناني (مترجمه),حزب الكتائب اللبنانية,أمين الجميل
2,-,-,قائمة عادية (مترجمه),أرئيل شارون
3,-,-,حزب الكتائب اللبنانية,بشير الجميل
4,-,-,مستقل (توضيح),رينيه معوض
5,-,-,حزب الكتائب اللبنانية,بشير الجميل
6,-,-,-,بوبي غودمان
7,-,لبناني (مترجمه),-,إيلي حبيقة
8,-,-,-,مجيد أرسلان
9,-,-,-,فادي أفرام


In [16]:
mkdir(datasets_dir)
df_politician_en.to_csv(os.path.join(datasets_dir, 'politicians_en.csv'), index=False)
df_politician_ar.to_csv(os.path.join(datasets_dir, 'politicians_ar.csv'), index=False, encoding='utf-8-sig')

In [17]:
df_locations_en = pd.DataFrame(columns=['Name', 'Subdivision type', 'Subdivision name'])
df_locations_ar = pd.DataFrame(columns=['اسم التقسيم'         , 'نوع التقسيم' , 'الاسم'])

In [18]:
for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'government_type' in page.data['infobox'] or 'subdivision_type' in page.data['infobox']:
                    print(page)
                    
                    if 'government_type' in page.data['infobox']:
                        name = get_list_items(page.data['infobox']['common_name']) if 'common_name' in page.data['infobox'] else  page.data['infobox']['conventional_long_name'] if 'conventional_long_name' in page.data['infobox'] else '-'
                    else:
                        name = get_list_items(page.data['infobox']['official_name']) if 'official_name' in page.data['infobox'] else page.data['infobox']['name'] if 'name' in  page.data['infobox'] else '-'

                    subdivision_type = get_list_items(page.data['infobox']['subdivision_type']) if 'subdivision_type' in page.data['infobox'] else '-'
                    subdivision_name = get_list_items(page.data['infobox']['subdivision_name']) if 'subdivision_name' in page.data['infobox'] else '-'
    
                    df_locations_en = df_locations_en.append({
                        'Name': ','.join(name) if isinstance(name, list) and len(name) > 1 else name,
                        'Subdivision type': ','.join(subdivision_type) if isinstance(subdivision_type, list) else subdivision_type,
                        'Subdivision name': ','.join(subdivision_name) if isinstance(subdivision_name, list) else subdivision_name                    
                    }, ignore_index=True)

                    print('-----------------')
                    print('name: ', name)
                    print('subdivision_type: ', subdivision_type)
                    print('subdivision_name: ', subdivision_name)
                    
                    
                    if isinstance(name, list):
                        name = name[0]
                    else:
                        pass
                    
                    if name != '-':
                        name_ar, name_ar_link = get_arabic_name(name)
                        print('name ar: ', name_ar)

                        if subdivision_type != '-':
                            print('START')
                            subdivision_type_ar = []
                            for st in subdivision_type:
                                st_ar, st_ar_link = get_arabic_name(st)
                                if st_ar != -1:
                                    print('subdivision_type: ', st)
                                    print('subdivision_type ar: ', st_ar),
                                    print('subdivision_type ar link: ', st_ar_link)
                                    subdivision_type_ar.append(st_ar)
                                else:
                                    continue
                            print('END')
                        else:
                            subdivision_type_ar = '-'
        

                        if subdivision_name != '-':
                            print('START')
                            subdivision_name_ar = []
                            for sn in subdivision_name:
                                sn_ar, sn_ar_link = get_arabic_name(sn)
                                if sn_ar != -1:
                                    print('subdivision_name: ', sn)
                                    print('subdivision_name ar: ', sn_ar),
                                    print('subdivision_name ar link: ', sn_ar_link)
                                    subdivision_name_ar.append(sn_ar)
                                else:
                                    continue
                            print('END')
                        else:
                            subdivision_name_ar = '-'

                        df_locations_ar = df_locations_ar.append({
                            'الاسم': ','.join(name_ar) if isinstance(name_ar, list) and len(name_ar) > 1 else name_ar,
                            'نوع التقسيم': ','.join(subdivision_type_ar) if isinstance(subdivision_type_ar, list) else subdivision_type_ar,
                            'اسم التقسيم': ','.join(subdivision_name_ar) if isinstance(subdivision_name_ar, list) else subdivision_name_ar,
                        }, ignore_index=True)
                    
                    else:
                        print('skipping {} because name is not found'.format(p))
        
        except LookupError:
            continue
                
            
    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 10th Airmobile Brigade (Lebanon)
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) 10th Infantry Brigade (Lebanon)
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) 11th Infantry Brigade (Lebanon)
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wik

-----------------
name:  ['Aabey']
subdivision_type:  ['Country']
subdivision_name:  ['Flag', 'Lebanon']
name ar:  آبي (مترجمه)
START
subdivision_type:  Country
subdivision_type ar:  بلد 
subdivision_type ar link:  https://ar.wikipedia.org/wiki/%D8%A8%D9%84%D8%AF
END
START
subdivision_name:  Flag
subdivision_name ar:  علم (راية) 
subdivision_name ar link:  https://ar.wikipedia.org/wiki/%D8%B9%D9%84%D9%85_(%D8%B1%D8%A7%D9%8A%D8%A9)
subdivision_name:  Lebanon
subdivision_name ar:  لبنان 
subdivision_name ar link:  https://ar.wikipedia.org/wiki/%D9%84%D8%A8%D9%86%D8%A7%D9%86
END


en.wikipedia.org (parse) Achrafieh


-----------------
name:  -
subdivision_type:  ['Country']
subdivision_name:  ['Lebanon', 'flagcountry']
skipping Achrafieh because name is not found


en.wikipedia.org (parse) Ain Ksour
en.wikipedia.org (parse) Air defense
en.wikipedia.org (parse) Aircraft carrier
en.wikipedia.org (parse) Al-Mourabitoun
en.wikipedia.org (parse) Al Bennay
en.wikipedia.org (parse) Aley


...

-----------------
name:  ['Beirut']
subdivision_type:  ['Country']
subdivision_name:  ['Lebanon']
name ar:  بيروت 
START
subdivision_type:  Country
subdivision_type ar:  بلد 
subdivision_type ar link:  https://ar.wikipedia.org/wiki/%D8%A8%D9%84%D8%AF
END
START
subdivision_name:  Lebanon
subdivision_name ar:  لبنان 
subdivision_name ar link:  https://ar.wikipedia.org/wiki/%D9%84%D8%A8%D9%86%D8%A7%D9%86
END


en.wikipedia.org (parse) Yarze


-----------------
name:  Yarze
subdivision_type:  ['Country']
subdivision_name:  ['Lebanon', 'flagcountry']
name ar:  يرزة 
START
subdivision_type:  Country
subdivision_type ar:  بلد 
subdivision_type ar link:  https://ar.wikipedia.org/wiki/%D8%A8%D9%84%D8%AF
END
START
subdivision_name:  Lebanon
subdivision_name ar:  لبنان 
subdivision_name ar link:  https://ar.wikipedia.org/wiki/%D9%84%D8%A8%D9%86%D8%A7%D9%86
Cannot find the link: http://en.wikipedia.org/wiki/flagcountry
subdivision_name:  flagcountry
subdivision_name ar:  بلد العلم (مترجمه)
subdivision_name ar link:  -1
END


en.wikipedia.org (parse) Yazbaki
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Youssef Tahan
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) ZPU
en.wikipedia.org (imageinfo) File:ZPU morrocan.jpg
en.wikipedia.org (parse) ZSU-23-4 Shilka
en.wikipedia.org (imageinfo) File:ЗСУ-23-4 «Шилка» в Музее отечес...
en.wikipedia.org (parse)

-----------------
name:  Zouk Mikael
subdivision_type:  ['Country']
subdivision_name:  ['Lebanon', 'flagcountry']
name ar:  ذوق مكايل 
START
subdivision_type:  Country
subdivision_type ar:  بلد 
subdivision_type ar link:  https://ar.wikipedia.org/wiki/%D8%A8%D9%84%D8%AF
END
START
subdivision_name:  Lebanon
subdivision_name ar:  لبنان 
subdivision_name ar link:  https://ar.wikipedia.org/wiki/%D9%84%D8%A8%D9%86%D8%A7%D9%86
Cannot find the link: http://en.wikipedia.org/wiki/flagcountry
subdivision_name:  flagcountry
subdivision_name ar:  بلد العلم (مترجمه)
subdivision_name ar link:  -1
END
skipping Template:Campaignbox Lebanese Civil War
skipping Template:Country data France
skipping Template:Country data Lebanon
skipping Template:Country data PLO
skipping Template:Country data Syria
skipping Template:Country data USA
skipping Template:Country data United States
skipping Template talk:Campaignbox Lebanese Civil War
skipping Category:Use dmy dates from July 2017


In [19]:
df_locations_en

,Name,Subdivision type,Subdivision name
0,[Aabey],Country,"Flag,Lebanon"
1,-,Country,"Lebanon,flagcountry"
2,[Aley],Country,"Flag,Lebanon"
3,[عاليه],Country,"Flag,Lebanon"
4,Aitat,Country,Lebanon
5,Baabda,Country,"Lebanon,flagcountry"
6,Boutchay District,Country,"Flag,Lebanon"
7,[Beirut],Country,Lebanon
8,Beit ed-Dine,Country,"Lebanon,flagcountry"
9,[Bhamdoun],Country,Lebanon


In [20]:
df_locations_ar

,اسم التقسيم,نوع التقسيم,الاسم
0,"علم (راية) ,لبنان",بلد,آبي (مترجمه)
1,"علم (راية) ,لبنان",بلد,عاليه (لبنان)
2,"علم (راية) ,لبنان",بلد,-1
3,لبنان,بلد,أب (مترجمه)
4,"لبنان ,بلد العلم (مترجمه)",بلد,بعبدا
5,"علم (راية) ,لبنان",بلد,منطقة Boutchay (مترجمه)
6,لبنان,بلد,بيروت
7,"لبنان ,بلد العلم (مترجمه)",بلد,بيت الدين
8,لبنان,بلد,بحمدون
9,"علم (راية) ,لبنان",بلد,بسوس


In [21]:
mkdir(datasets_dir)
df_locations_en.to_csv(os.path.join(datasets_dir, 'locations_en.csv'), index=False)
df_locations_ar.to_csv(os.path.join(datasets_dir, 'locations_ar.csv'), index=False, encoding='utf-8-sig')